# 构建一个提取链

在本教程中，我们将使用 [聊天模型](/docs/concepts/chat_models) 的 [工具调用](/docs/concepts/tool_calling) 功能从非结构化文本中提取结构化信息。我们还将演示如何在此场景中使用 [少样本提示](/docs/concepts/few_shot_prompting/) 来提高性能。

:::important
此教程需要 `langchain-core>=0.3.20`，并且仅适用于支持 **工具调用** 的模型。
:::

## 设置

### Jupyter Notebook

本教程以及其他教程，也许在 [Jupyter notebooks](https://jupyter.org/) 中运行是最方便的。在交互式环境中学习指南是更好地理解它们的好方法。有关安装说明，请参见 [此处](https://jupyter.org/install)。

### 安装

要安装 LangChain，请运行：

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Tabs>
  <TabItem value="pip" label="Pip" default>
    <CodeBlock language="bash">pip install --upgrade langchain-core</CodeBlock>
  </TabItem>
  <TabItem value="conda" label="Conda">
    <CodeBlock language="bash">conda install langchain-core -c conda-forge</CodeBlock>
  </TabItem>
</Tabs>



有关更多详细信息，请参阅我们的 [安装指南](/docs/how_to/installation)。

### LangSmith

您使用 LangChain 构建的许多应用程序将包含多个步骤和对 LLM 的多次调用。
随着这些应用程序变得越来越复杂，能够检查链或代理内部究竟发生了什么就变得至关重要。
做到这一点最好方法是使用 [LangSmith](https://smith.langchain.com)。

在上面的链接注册后，请确保设置您的环境变量以开始记录跟踪：

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```

或者，如果您在 notebook 中，可以使用以下方式设置：

```python
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
```

## Schema

首先，我们需要描述我们想从文本中提取哪些信息。

我们将使用 Pydantic 来定义一个示例 Schema，用于提取个人信息。

In [2]:
from typing import Optional

from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )

定义 schema 时，有两个最佳实践：

1.  记录 **属性** 和 **schema** 本身：此信息将发送给 LLM，用于提高信息提取的质量。
2.  不要强迫 LLM 编造信息！我们在上面使用了 `Optional` 来定义属性，允许 LLM 在不知道答案时输出 `None`。

:::important
为达到最佳性能，请务必充分记录 schema，并确保模型在文本中没有可提取的信息时，不会被强迫返回结果。
:::

## Extractor

让我们使用上面定义的 schema 来创建一个信息提取器。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

我们需要使用一个支持函数/工具调用的模型。

请参阅[文档](/docs/concepts/tool_calling)了解所有可与此 API 结合使用的模型。

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />

In [4]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [7]:
structured_llm = llm.with_structured_output(schema=Person)

让我们来测试一下：

In [8]:
text = "Alan Smith is 6 feet tall and has blond hair."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Person(name='Alan Smith', hair_color='blond', height_in_meters='1.83')

:::important

提取是生成式的 🤯

LLM 是生成模型，因此它们可以做一些很酷的事情，例如正确地提取人的身高（米），即使它原本是以英尺提供的！
:::

我们可以在[这里](https://smith.langchain.com/public/44b69a63-3b3b-47b8-8a6d-61b46533f015/r)查看 LangSmith trace。请注意，[trace 的聊天模型部分](https://smith.langchain.com/public/44b69a63-3b3b-47b8-8a6d-61b46533f015/r/dd1f6305-f1e9-4919-bd8f-339d03a12d01)显示了发送给模型的精确消息序列、调用的工具以及其他元数据。

## 多个实体

在**大多数情况**下，你应该提取一系列实体，而不是单个实体。

通过将模型相互嵌套，可以使用 pydantic 轻松实现这一点。

In [9]:
from typing import List, Optional

from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )


class Data(BaseModel):
    """Extracted data about people."""

    # Creates a model so that we can extract multiple entities.
    people: List[Person]

:::important
提取结果可能不尽完美。请继续阅读，了解如何使用 **Reference Examples** 来提高提取质量，并参阅我们的提取 [操作指南](/docs/how_to/#extraction) 以获取更多详细信息。
:::

In [10]:
structured_llm = llm.with_structured_output(schema=Data)
text = "My name is Jeff, my hair is black and i am 6 feet tall. Anna has the same color hair as me."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Data(people=[Person(name='Jeff', hair_color='black', height_in_meters='1.83'), Person(name='Anna', hair_color='black', height_in_meters=None)])

:::tip
当 Schema 能够提取**多个实体**时，它也允许模型在文本中没有相关信息的情况下提取**零个实体**，只需提供一个空列表即可。

这通常是一件**好事**！它允许我们在一个实体上指定**必需**的属性，而不必强制模型检测到该实体。
:::

我们可以在这里查看 LangSmith 的追踪记录：[here](https://smith.langchain.com/public/7173764d-5e76-45fe-8496-84460bd9cdef/r)。

## 参考示例

LLM 应用的行为可以通过 [少样本提示 (few-shot prompting)](/docs/concepts/few_shot_prompting/) 进行引导。对于 [聊天模型 (chat models)](/docs/concepts/chat_models/) 来说，这可以表现为一系列的输入和响应消息对，以展示期望的行为。

例如，我们可以通过交替出现的 `user` 和 `assistant` [消息 (messages)](/docs/concepts/messages/#role) 来传达一个符号的含义：

In [11]:
messages = [
    {"role": "user", "content": "2 🦜 2"},
    {"role": "assistant", "content": "4"},
    {"role": "user", "content": "2 🦜 3"},
    {"role": "assistant", "content": "5"},
    {"role": "user", "content": "3 🦜 4"},
]

response = llm.invoke(messages)
print(response.content)

7


[结构化输出](/docs/concepts/structured_outputs/) 通常会在底层使用 [工具调用](/docs/concepts/tool_calling/)。这通常涉及生成包含工具调用的 [AI 消息](/docs/concepts/messages/#aimessage)，以及包含工具调用结果的 [工具消息](/docs/concepts/messages/#toolmessage)。在这种情况下，消息序列应该是什么样的？

不同的 [聊天模型提供商](/docs/integrations/chat/) 对有效的消息序列有不同的要求。有些接受如下（重复的）消息序列：

- 用户消息
- 包含工具调用的 AI 消息
- 包含结果的工具消息

另一些则需要一个最终的 AI 消息来包含某种响应。

LangChain 包含一个实用函数 [tool_example_to_messages](https://python.langchain.com/api_reference/core/utils/langchain_core.utils.function_calling.tool_example_to_messages.html)，它能为大多数模型提供商生成有效的序列。该函数通过仅要求 Pydantic 表示法来简化结构化少样本示例的生成。

让我们试一试。我们可以将输入字符串对和期望的 Pydantic 对象转换为一系列消息，提供给聊天模型。在底层，LangChain 会将工具调用格式化为每个提供商所需 的格式。

请注意：此版本的 `tool_example_to_messages` 需要 `langchain-core>=0.3.20`。

In [13]:
from langchain_core.utils.function_calling import tool_example_to_messages

examples = [
    (
        "The ocean is vast and blue. It's more than 20,000 feet deep.",
        Data(people=[]),
    ),
    (
        "Fiona traveled far from France to Spain.",
        Data(people=[Person(name="Fiona", height_in_meters=None, hair_color=None)]),
    ),
]


messages = []

for txt, tool_call in examples:
    if tool_call.people:
        # This final message is optional for some providers
        ai_response = "Detected people."
    else:
        ai_response = "Detected no people."
    messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))

检查结果时，我们会看到这两个示例对生成了八条消息：

In [12]:
for message in messages:
    message.pretty_print()

================================ Human Message =================================

The ocean is vast and blue. It's more than 20,000 feet deep.
================================== Ai Message ==================================
Tool Calls:
  Data (d8f2e054-7fb9-417f-b28f-0447a775b2c3)
 Call ID: d8f2e054-7fb9-417f-b28f-0447a775b2c3
  Args:
    people: []
================================= Tool Message =================================

You have correctly called this tool.
================================== Ai Message ==================================

Detected no people.
================================ Human Message =================================

Fiona traveled far from France to Spain.
================================== Ai Message ==================================
Tool Calls:
  Data (0178939e-a4b1-4d2a-a93e-b87f665cdfd6)
 Call ID: 0178939e-a4b1-4d2a-a93e-b87f665cdfd6
  Args:
    people: [{'name': 'Fiona', 'hair_color': None, 'height_in_meters': None}]
================================

让我们来比较一下是否包含这些消息时的性能。例如，让我们传递一个我们不打算提取任何人的消息：

In [15]:
message_no_extraction = {
    "role": "user",
    "content": "The solar system is large, but earth has only 1 moon.",
}

structured_llm = llm.with_structured_output(schema=Data)
structured_llm.invoke([message_no_extraction])

Data(people=[Person(name='Earth', hair_color='None', height_in_meters='0.00')])

在此示例中，模型可能会错误地生成人物记录。

由于我们的少样本示例包含“负面”示例，因此我们鼓励模型在这种情况下表现正确：

In [16]:
structured_llm.invoke(messages + [message_no_extraction])

Data(people=[])

:::tip

[LangSmith](https://smith.langchain.com/public/b3433f57-7905-4430-923c-fed214525bf1/r) 的运行追踪记录显示了发送到聊天模型的准确消息顺序、生成的工具调用、延迟、Token 数量以及其他元数据。

:::

有关提取工作流和参考示例的更多详细信息，请参阅[本指南](/docs/how_to/extraction_examples/)，其中包括如何整合 Prompt 模板和自定义示例消息的生成。

## 后续步骤

既然您已经了解了 LangChain 的提取基础知识，就可以继续阅读其余的操作指南了：

- [添加示例](/docs/how_to/extraction_examples)：关于使用**参考示例**来提高性能的更多详细信息 。
- [处理长文本](/docs/how_to/extraction_long_text)：如果文本不适合 LLM 的上下文窗口，您应该怎么做？
- [使用解析方法](/docs/how_to/extraction_parse)：对于不支持**工具/函数调用**的模型，使用基于提示的方法进行提取。